In [1]:
from marubatsu import Marubatsu
from ai import ai14s

mb = Marubatsu()
ai14s(mb, debug=True)

Start ai_by_score
Turn o
...
...
...

legal_moves [(0, 0), (1, 0), (2, 0), (0, 1), (1, 1), (2, 1), (0, 2), (1, 2), (2, 2)]
move (0, 0)
Turn x
O..
...
...

defaultdict(<class 'int'>,
            {Markpat(last_turn=0, turn=0, empty=3): 5,
             Markpat(last_turn=1, turn=0, empty=2): 3})
score 1.5 best score -inf
UPDATE
  best score 1.5
  best moves [(0, 0)]
move (1, 0)
Turn x
.O.
...
...

defaultdict(<class 'int'>,
            {Markpat(last_turn=0, turn=0, empty=3): 6,
             Markpat(last_turn=1, turn=0, empty=2): 2})
score 1.0 best score 1.5
move (2, 0)
Turn x
..O
...
...

defaultdict(<class 'int'>,
            {Markpat(last_turn=0, turn=0, empty=3): 5,
             Markpat(last_turn=1, turn=0, empty=2): 3})
score 1.5 best score 1.5
APPEND
  best moves [(0, 0), (2, 0)]
move (0, 1)
Turn x
...
O..
...

defaultdict(<class 'int'>,
            {Markpat(last_turn=0, turn=0, empty=3): 6,
             Markpat(last_turn=1, turn=0, empty=2): 2})
score 1.0 best score 1.5
move (1, 1)
T

(1, 1)

In [2]:
def ai3s(mb, debug=False, analyze=False):
    def eval_func(mb):
        if mb.last_move == (1, 1):
            return 1
        else:
            return 0
        
    return ai_by_score(mb, eval_func, debug=debug, analyze=analyze)


In [3]:
from copy import deepcopy
from random import choice
from ai import dprint

def ai_by_score(mb_orig, eval_func, debug=False, rand=True, analyze=False):   
    dprint(debug, "Start ai_by_score")
    dprint(debug, mb_orig)
    legal_moves = mb_orig.calc_legal_moves()
    dprint(debug, "legal_moves", legal_moves)
    best_score = float("-inf")
    best_moves = []
    if analyze:
        score_by_move = {}
    for move in legal_moves:
        dprint(debug, "=" * 20)
        dprint(debug, "move", move)
        mb = deepcopy(mb_orig)
        x, y = move
        mb.move(x, y)
        dprint(debug, mb)
        
        score = eval_func(mb)
        dprint(debug, "score", score, "best score", best_score)
        if analyze:
            score_by_move[move] = score
        
        if best_score < score:
            best_score = score
            best_moves = [move]
            dprint(debug, "UPDATE")
            dprint(debug, "  best score", best_score)
            dprint(debug, "  best moves", best_moves)
        elif best_score == score:
            best_moves.append(move)
            dprint(debug, "APPEND")
            dprint(debug, "  best moves", best_moves)

    dprint(debug, "=" * 20)
    dprint(debug, "Finished")
    dprint(debug, "best score", best_score)
    dprint(debug, "best moves", best_moves)
    if analyze:
        return {
            "candidate": best_moves,
            "score_by_move": score_by_move,
        }
    elif rand:   
        return choice(best_moves)
    else:
        return best_moves[0]

In [4]:
from pprint import pprint
 
pprint(ai3s(mb, analyze=True))

{'candidate': [(1, 1)],
 'score_by_move': {(0, 0): 0,
                   (0, 1): 0,
                   (0, 2): 0,
                   (1, 0): 0,
                   (1, 1): 1,
                   (1, 2): 0,
                   (2, 0): 0,
                   (2, 1): 0,
                   (2, 2): 0}}


In [5]:
mb.move(1, 1)
pprint(ai3s(mb, analyze=True))

{'candidate': [(0, 0), (1, 0), (2, 0), (0, 1), (2, 1), (0, 2), (1, 2), (2, 2)],
 'score_by_move': {(0, 0): 0,
                   (0, 1): 0,
                   (0, 2): 0,
                   (1, 0): 0,
                   (1, 2): 0,
                   (2, 0): 0,
                   (2, 1): 0,
                   (2, 2): 0}}


In [6]:
def ai3(mb, analyze=False):
    if mb.board[1][1] == Marubatsu.EMPTY:
        candidate = [(1, 1)]
    else:
        candidate = mb.calc_legal_moves()
    if analyze:
        return {
            "candidate": candidate,
            "score_by_move": None
        }
    else:
        return choice(candidate)

In [7]:
mb.restart()
pprint(ai3(mb, analyze=True))

{'candidate': [(1, 1)], 'score_by_move': None}


In [8]:
mb.move(1, 1)
pprint(ai3(mb, analyze=True))

{'candidate': [(0, 0), (1, 0), (2, 0), (0, 1), (2, 1), (0, 2), (1, 2), (2, 2)],
 'score_by_move': None}


In [9]:
from util import Check_solved, load_bestmoves, load_mblist
from tqdm import tqdm

@staticmethod
def is_strongly_solved(ai, params=None, consider_samedata=True):
    if Check_solved.bestmoves_by_board is None:
        Check_solved.bestmoves_by_board = load_bestmoves ("../data/bestmoves_by_board.dat")
    if consider_samedata:
        if Check_solved.mblist_by_board_min is None:
            Check_solved.mblist_by_board_min = load_mblist("../data/mblist_by_board_min.dat")
        mblist = Check_solved.mblist_by_board_min
    else:
        if Check_solved.mblist_by_board2 is None:
            Check_solved.mblist_by_board2 = load_mblist("../data/mblist_by_board2.dat") 
        mblist = Check_solved.mblist_by_board2        
    if params is None:
        params = {}
    count = 0
    incorrectlist = []
    for mb in tqdm(mblist):
        candidate = set(ai(mb, analyze=True, **params)["candidate"])
        bestmoves = set(Check_solved.bestmoves_by_board[mb.board_to_str()])
        if candidate <= bestmoves:
            count += 1
        else:
            incorrectlist.append((mb, candidate, bestmoves))
    nodenum = len(mblist)
    print(f"{count}/{nodenum} {count/nodenum*100:.2f}%")
    return count == nodenum, incorrectlist

Check_solved.is_strongly_solved = is_strongly_solved

In [10]:
@staticmethod
def is_weakly_solved_r(node, ai, turn, params, registered_boards):
    txt = node.mb.board_to_str()
    if txt in registered_boards:
        return True
    Check_solved.count += 1
    registered_boards.add(txt)
    if node.mb.status == turn or node.mb.status == Marubatsu.DRAW:
        return True
    elif node.mb.status != Marubatsu.PLAYING:
        return False
        
    if turn == node.mb.turn:
        moves = ai(node.mb, analyze=True, **params)["candidate"]
    else:
        moves = node.mb.calc_legal_moves()
    for move in moves:
        childnode = node.children_by_move[move]
        if not Check_solved.is_weakly_solved_r(childnode, ai, turn, params, registered_boards):
            return False
    return True 

Check_solved.is_weakly_solved_r = is_weakly_solved_r

In [11]:
Check_solved.is_strongly_solved(ai3s)

100%|██████████| 431/431 [00:00<00:00, 3315.25it/s]

117/431 27.15%


(False,
 [(<marubatsu.Marubatsu at 0x286603756d0>, {(1, 2), (2, 1), (2, 2)}, {(2, 2)}),
  (<marubatsu.Marubatsu at 0x286603759d0>, {(1, 2), (2, 2)}, {(2, 2)}),
  (<marubatsu.Marubatsu at 0x28660375cd0>, {(2, 1), (2, 2)}, {(2, 2)}),
  (<marubatsu.Marubatsu at 0x28660375fd0>, {(0, 2), (1, 2), (2, 1)}, {(0, 2)}),
  (<marubatsu.Marubatsu at 0x286603762d0>, {(0, 2), (1, 2)}, {(0, 2)}),
  (<marubatsu.Marubatsu at 0x286603765d0>, {(0, 2), (2, 1)}, {(0, 2)}),
  (<marubatsu.Marubatsu at 0x286603768d0>, {(1, 1)}, {(2, 2)}),
  (<marubatsu.Marubatsu at 0x28660376bd0>, {(0, 2), (1, 2), (2, 2)}, {(2, 2)}),
  (<marubatsu.Marubatsu at 0x28660376e90>, {(1, 2), (2, 2)}, {(1, 2)}),
  (<marubatsu.Marubatsu at 0x28660377190>, {(0, 2), (2, 2)}, {(2, 2)}),
  (<marubatsu.Marubatsu at 0x28660377490>, {(1, 1)}, {(2, 2)}),
  (<marubatsu.Marubatsu at 0x28660377790>, {(1, 1)}, {(2, 2)}),
  (<marubatsu.Marubatsu at 0x286603792d0>, {(0, 2), (2, 1), (2, 2)}, {(2, 1)}),
  (<marubatsu.Marubatsu at 0x28660379590>, {(0, 

In [12]:
Check_solved.is_weakly_solved(ai3s)

   o False
   x False
Both False


False

In [13]:
Check_solved.is_strongly_solved(ai3)

100%|██████████| 431/431 [00:00<00:00, 8980.35it/s]

117/431 27.15%


(False,
 [(<marubatsu.Marubatsu at 0x286603756d0>, {(1, 2), (2, 1), (2, 2)}, {(2, 2)}),
  (<marubatsu.Marubatsu at 0x286603759d0>, {(1, 2), (2, 2)}, {(2, 2)}),
  (<marubatsu.Marubatsu at 0x28660375cd0>, {(2, 1), (2, 2)}, {(2, 2)}),
  (<marubatsu.Marubatsu at 0x28660375fd0>, {(0, 2), (1, 2), (2, 1)}, {(0, 2)}),
  (<marubatsu.Marubatsu at 0x286603762d0>, {(0, 2), (1, 2)}, {(0, 2)}),
  (<marubatsu.Marubatsu at 0x286603765d0>, {(0, 2), (2, 1)}, {(0, 2)}),
  (<marubatsu.Marubatsu at 0x286603768d0>, {(1, 1)}, {(2, 2)}),
  (<marubatsu.Marubatsu at 0x28660376bd0>, {(0, 2), (1, 2), (2, 2)}, {(2, 2)}),
  (<marubatsu.Marubatsu at 0x28660376e90>, {(1, 2), (2, 2)}, {(1, 2)}),
  (<marubatsu.Marubatsu at 0x28660377190>, {(0, 2), (2, 2)}, {(2, 2)}),
  (<marubatsu.Marubatsu at 0x28660377490>, {(1, 1)}, {(2, 2)}),
  (<marubatsu.Marubatsu at 0x28660377790>, {(1, 1)}, {(2, 2)}),
  (<marubatsu.Marubatsu at 0x286603792d0>, {(0, 2), (2, 1), (2, 2)}, {(2, 1)}),
  (<marubatsu.Marubatsu at 0x28660379590>, {(0, 

In [14]:
Check_solved.is_weakly_solved(ai3)

   o False
   x False
Both False


False

In [15]:
from util import gui_play

gui_play()

In [16]:
from marubatsu import Marubatsu_GUI

def update_gui(self):
    def calc_status_txt(score):
        if score > 0:
            return "〇"
        elif score == 0:
            return "△"
        else:
            return "×"
    
    ax = self.ax

    # Axes の内容をクリアして、これまでの描画内容を削除する
    ax.clear()

    # y 軸を反転させる
    ax.invert_yaxis()

    # 枠と目盛りを表示しないようにする
    ax.axis("off")   

    # リプレイ中、ゲームの決着がついていた場合は背景色を変更する
    is_replay =  self.mb.move_count < len(self.mb.records) - 1 
    if self.mb.status == Marubatsu.PLAYING:
        facecolor = "lightcyan" if is_replay else "white"
    else:
        facecolor = "lightyellow"

    ax.figure.set_facecolor(facecolor)
        
    # 上部のメッセージを描画する
    # 対戦カードの文字列を計算する
    ax.text(1.5, 3.5, f"{self.dropdown_list[0].label}　VS　{self.dropdown_list[1].label}", 
            fontsize=7*self.size, ha="center")   

    # ゲームの決着がついていない場合は、手番を表示する
    if self.mb.status == Marubatsu.PLAYING:
        text = "Turn " + self.mb.turn
        score = self.score_table[self.mb.board_to_str()]["score"]
        if self.show_status:
            text += " " + calc_status_txt(score)
    # 引き分けの場合
    elif self.mb.status == Marubatsu.DRAW:
        text = "Draw game"
    # 決着がついていれば勝者を表示する
    else:
        text = "Winner " + self.mb.status
    # リプレイ中の場合は "Replay" を表示する
    if is_replay:
        text += " Replay"
    ax.text(0, -0.2, text, fontsize=7*self.size)

    self.draw_board(ax, self.mb, lw=0.7*self.size)
    
    if self.show_status:
        bestmoves = self.score_table[self.mb.board_to_str()]["bestmoves"]
        analyze = ai3s(self.mb, analyze=True)
        score_by_move = analyze["score_by_move"]
        candidate = analyze["candidate"]
        for move in self.mb.calc_legal_moves():
            x, y = move
            mb = deepcopy(self.mb)
            mb.move(x, y)
            score = self.score_table[mb.board_to_str()]["score"]
            color = "red" if move in bestmoves else "black"
            text = calc_status_txt(score)
            ax.text(x + 0.1, y + 0.35, text, fontsize=5*self.size, c=color)
            if score_by_move is not None:
                color = "red" if move in candidate else "black"
                ax.text(x + 0.1, y + 0.65, score_by_move[move], fontsize=5*self.size, c=color)
            elif move in candidate:
                ax.text(x + 0.1, y + 0.65, "候補手", fontsize=4.8*self.size)
                
    self.update_widgets_status()

    if hasattr(self, "mbtree_gui"):
        from tree import Node

        self.mbtree_gui.selectednode = Node(self.mb, depth=self.mb.move_count)
        self.mbtree_gui.update_gui()
        
Marubatsu_GUI.update_gui = update_gui

In [17]:
gui_play()

In [18]:
def update_gui(self):
    def calc_status_txt(score):
        if score > 0:
            return "〇"
        elif score == 0:
            return "△"
        else:
            return "×"
    
    ax = self.ax

    # Axes の内容をクリアして、これまでの描画内容を削除する
    ax.clear()

    # y 軸を反転させる
    ax.invert_yaxis()

    # 枠と目盛りを表示しないようにする
    ax.axis("off")   

    # リプレイ中、ゲームの決着がついていた場合は背景色を変更する
    is_replay =  self.mb.move_count < len(self.mb.records) - 1 
    if self.mb.status == Marubatsu.PLAYING:
        facecolor = "lightcyan" if is_replay else "white"
    else:
        facecolor = "lightyellow"

    ax.figure.set_facecolor(facecolor)
        
    # 上部のメッセージを描画する
    # 対戦カードの文字列を計算する
    ax.text(1.5, 3.5, f"{self.dropdown_list[0].label}　VS　{self.dropdown_list[1].label}", 
            fontsize=7*self.size, ha="center")   

    # ゲームの決着がついていない場合は、手番を表示する
    if self.mb.status == Marubatsu.PLAYING:
        text = "Turn " + self.mb.turn
        score = self.score_table[self.mb.board_to_str()]["score"]
        if self.show_status:
            text += " " + calc_status_txt(score)
    # 引き分けの場合
    elif self.mb.status == Marubatsu.DRAW:
        text = "Draw game"
    # 決着がついていれば勝者を表示する
    else:
        text = "Winner " + self.mb.status
    # リプレイ中の場合は "Replay" を表示する
    if is_replay:
        text += " Replay"
    ax.text(0, -0.2, text, fontsize=7*self.size)

    self.draw_board(ax, self.mb, lw=0.7*self.size)
    
    if self.show_status:
        bestmoves = self.score_table[self.mb.board_to_str()]["bestmoves"]
        analyze = ai3(self.mb, analyze=True)
        score_by_move = analyze["score_by_move"]
        candidate = analyze["candidate"]
        for move in self.mb.calc_legal_moves():
            x, y = move
            mb = deepcopy(self.mb)
            mb.move(x, y)
            score = self.score_table[mb.board_to_str()]["score"]
            color = "red" if move in bestmoves else "black"
            text = calc_status_txt(score)
            ax.text(x + 0.1, y + 0.35, text, fontsize=5*self.size, c=color)
            if score_by_move is not None:
                color = "red" if move in candidate else "black"
                ax.text(x + 0.1, y + 0.65, score_by_move[move], fontsize=5*self.size, c=color)
            elif move in candidate:
                ax.text(x + 0.1, y + 0.65, "候補手", fontsize=4.8*self.size)
                
    self.update_widgets_status()

    if hasattr(self, "mbtree_gui"):
        from tree import Node

        self.mbtree_gui.selectednode = Node(self.mb, depth=self.mb.move_count)
        self.mbtree_gui.update_gui()
        
Marubatsu_GUI.update_gui = update_gui

In [19]:
gui_play()